In [ ]:
#r "nuget: xunit, 2.8.1"
using System;
using System.Threading;
using System.Linq;
using Xunit;

class DefiniteIntegral
{
    //
    // a, b - границы отрезка, на котором происходит вычисление опредленного интеграла
    // function - функция, для которой вычисляется определнный интеграл
    // step - размер одного шага разбиения
    // threadsnumber - число потоков, которые используются для вычислений
    //
    public static double Solve(double a, double b, Func<double, double> function, double step, int threadsnumber)
    {
       long resultat = 0;
       Barrier barrier = new Barrier(threadsnumber);    
    Thread[] threads = Enumerable.Range(0, threadsnumber).Select(index =>
            {
                Thread thread = new Thread(() =>
                {
                    double tResultat = 0.0;
                    double start = a + index * ((b - a) / threadsnumber);
                    double end = start + ((b - a) / threadsnumber);

                    for (double i = start; i < end; i += step)
                    {
                        tResultat += (function(i) + function(i + step)) * step *0.5;
                    }

                    Interlocked.Add(ref resultat, (long)(tResultat * 1e5));
                });

                thread.Start();
                return thread;
            })
            .ToArray();

        foreach (Thread thread in threads)
        {
            thread.Join();
        }
      
        return Math.Round(resultat / 1e5);
    }
}

In [ ]:
var X = (double x) => x;
Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2), 1e-4);

In [ ]:
var SIN = (double x) => Math.Sin(x);
Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8), 1e-4);

In [ ]:
var X = (double x) => x;
Assert.Equal(50, DefiniteIntegral.Solve(0, 10, X, 1e-6, 8), 1e-5);